In [312]:
import QuantLib as ql
import pandas as pd

In [313]:
calc_date = ql.Date(12, 4, 2017)
ql.Settings.instance().evaluationDate = calc_date
ytm = ql.SimpleQuote(0.07431916541307615)
dates = [ql.Date(12,4,2017), ql.Date(12,4,2018), ql.Date(12,4,2019)]
zeros = [0.07431916541307615,0.07431916541307615,0.07431916541307615]
# spot_curve = ql.ZeroCurve(dates, zeros, ql.ActualActual(), ql.TARGET(), ql.Linear(), ql.Semiannual)
spot_curve = ql.FlatForward(calc_date, ql.QuoteHandle(ytm), ql.Actual365Fixed())
spot_curve_handle = ql.YieldTermStructureHandle(spot_curve)

issue_date = ql.Date(21, 12, 2016)
maturity_date = ql.Date(21, 12, 2018)
tenor = ql.Period(ql.Semiannual)
calendar = ql.SouthAfrica()
business_convention = ql.Following
date_generation = ql.DateGeneration.Backward
month_end = False
schedule = ql.Schedule(issue_date, maturity_date, tenor,
    calendar, business_convention,
    business_convention, date_generation,
    month_end)

In [314]:
coupon_rate = .08
coupons = [coupon_rate]
settlement_days = 0
face_value = 100
day_count = ql.Actual365Fixed()
#day_count = ql.SimpleDayCounter()
fixed_rate_bond = ql.FixedRateBond(settlement_days,
    face_value,
    schedule,
    coupons,
    day_count)

In [315]:
bond_engine = ql.DiscountingBondEngine(spot_curve_handle)
fixed_rate_bond.setPricingEngine(bond_engine)

In [316]:
p = fixed_rate_bond.NPV()
p

103.10594581468344

In [317]:
fixed_rate_bond.accruedAmount()

2.4547945205479538

In [318]:
fixed_rate_bond.cleanPrice()

100.65115129413549

In [319]:
fixed_rate_bond.dirtyPrice()

103.10594581468344

In [320]:
price_check = (fixed_rate_bond.NPV()/103.11510-1)*100
price_check

-0.00887763801475927

In [321]:
pd.DataFrame([ (c.date(), c.amount())
    for c in fixed_rate_bond.cashflows() ],
    columns = ('date', 'amount'),
    index=['']*len(fixed_rate_bond.cashflows()))

,date,amount
,"June 21st, 2017",3.989041
,"December 21st, 2017",4.010959
,"June 21st, 2018",3.989041
,"December 21st, 2018",4.010959
,"December 21st, 2018",100.000000


In [322]:
spread = 0.07431916541307615-0.01
ytm.setValue(spread)
pu = fixed_rate_bond.NPV()

spread = 0.07431916541307615+0.01
ytm.setValue(spread)
pd = fixed_rate_bond.NPV()

In [323]:
mac_duration = (pu-pd)/(2*p*0.01)
mac_duration

1.5812584476414542